In [1]:
import sys
import os
import subprocess

from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession, HiveContext ,Row
from pyspark.sql import functions as F
from datetime import date
from dateutil.relativedelta import relativedelta
from pyspark.sql.window import Window
import numpy as np
import datetime as dt
from dateutil.relativedelta import relativedelta
from pyspark.sql.window import Window
from pyspark.sql.functions import sequence, col, expr, rand, when, lit, concat

from datetime import datetime
import pandas as pd
import random
from datetime import timedelta
import random

from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.window import Window
import random
from datetime import datetime, timedelta
from pyspark.sql.types import StructType, StructField, DateType, IntegerType

In [2]:
spark = SparkSession.builder \
                .appName("spark_session") \
                .config("spark.sql.catalogImplementation", "hive") \
                .config("spark.sql.caseSensitive", "false") \
                .config("hive.exec.dynamic.partition", "true") \
                .config("hive.exec.dynamic.partition.mode", "nonstrict") \
                .config("spark.sql.htl.check", "false") \
                .config("spark.sql.legacy.allowCreatingManagedTableUsingNonmptyLocation", "true") \
                .enableHiveSupport() \
                .master("local[*]") \
                .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/09/12 18:05:22 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/09/12 18:05:23 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


#### Load config:

In [19]:
# ==========================================
# Load config
# ==========================================

#config = spark.table("db.xs0_config")

config = spark.read.parquet('xs0_config.parquet')

time_series_prep_sdate = config.collect()[0][0]
time_series_prep_sdate = config.collect()[0][0]
time_series_prep_edate = config.collect()[0][1]
train_start_date = config.collect()[0][2]
train_end_date = config.collect()[0][3]
test_start_date = config.collect()[0][4]
test_end_date = config.collect()[0][5]
score_date = config.collect()[0][6]
model_version_prefix = config.collect()[0][7]

config.show()

+----------------------+----------------------+----------------+--------------+---------------+-------------+----------+--------------------+
|time_series_prep_sdate|time_series_prep_edate|train_start_date|train_end_date|test_start_date|test_end_date|score_date|model_version_prefix|
+----------------------+----------------------+----------------+--------------+---------------+-------------+----------+--------------------+
|            2021-04-01|            2023-09-01|      2022-06-01|    2023-06-01|     2023-07-01|   2023-09-01|2023-10-01|            20230831|
+----------------------+----------------------+----------------+--------------+---------------+-------------+----------+--------------------+



## Customer 360

In [12]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.window import Window
import random
from datetime import datetime, timedelta
from pyspark.sql.types import StructType, StructField, DateType

# Define the number of rows in the dataset
num_rows = 50000

# Define the range of __PERIOD values
start_date = datetime(2021, 4, 1)
end_date = datetime(2023, 9, 1)

# Create a list of __PERIOD values by adding months manually
date_range = []
current_date = start_date
while current_date <= end_date:
    date_range.append(current_date)
    year = current_date.year + (current_date.month // 12)
    month = current_date.month % 12 + 1
    current_date = datetime(year, month, 1)

# Define the schema for the DataFrame
schema = StructType([StructField("__PERIOD", DateType(), False)])

# Create a DataFrame with __PERIOD values and the specified schema
period_df = spark.createDataFrame([(d,) for d in date_range], schema)

# Generate random customer_no values
customer_no_values = [str(random.randint(1, num_rows)) for _ in range(num_rows)]

# Create a DataFrame for customer_no
customer_no_df = spark.createDataFrame([(customer_no,) for customer_no in customer_no_values], ["customer_no"])

# Cross-join customer_no_df and period_df to create unique combinations
customer_df = customer_no_df.crossJoin(period_df)

# Generate other customer characteristics features
customer_df = customer_df \
    .withColumn('id_gender', expr('CAST(RAND() > 0.5 AS INT)')) \
    .withColumn('customer_category_cleaned', expr('CAST(RAND() * 3 + 1 AS INT)')) \
    .withColumn('marital_status_cleaned', expr('CAST(RAND() > 0.5 AS INT)')) \
    .withColumn('customer_governorate', expr('CAST(RAND() * 10 + 1 AS INT)')) \
    .withColumn('age', expr('CAST(RAND() * 67 + 18 AS INT)')) \
    .withColumn('job_cleaned', expr('CAST(RAND() * 10 + 1 AS INT)')) \
    .withColumn('local_branch', expr('CAST(RAND() * 100 + 1 AS INT)')) \
    .withColumn('employed', expr('CAST(RAND() > 0.5 AS INT)')) \
    .withColumn('tenure', expr('CAST(RAND() * 804 AS INT)')) \
    .withColumn('customer_active_SOP', expr('CAST(RAND() <= 0.95 AS INT)'))

# Generate Cross-sell and up-sell targets
products = ['savacc', 'currentacc', 'deposits', 'bank_online', 'credit_cards', 'insurance', 'debit_cards', 'secured_loans', 'unsecured_loans']

for product in products:
    customer_df = customer_df.withColumn(f'{product}__new_in_period', expr(f'CAST(RAND() <= 0.05 AS INT)'))
    customer_df = customer_df.withColumn(f'{product}__close_in_period', expr(f'CASE WHEN {product}__new_in_period == 1 THEN 0 ELSE CAST(RAND() <= 0.05 AS INT) END'))
    customer_df = customer_df.withColumn(f'{product}__active_SOP', expr(f'CASE WHEN {product}__close_in_period == 1 THEN 0 ELSE CAST(RAND() <= 0.95 AS INT) END'))
    customer_df = customer_df.withColumn(f'{product}__time_to_next_maturity', when(col(f'{product}__active_SOP') == 1, expr('CAST(RAND() * 3650 AS INT)')).otherwise(0))
    customer_df = customer_df.withColumn(f'{product}__active_at_or_before_SOP', when(col(f'{product}__active_SOP') == 1, 1).otherwise(0))
    customer_df = customer_df.withColumn(f'{product}__time_since_last_new', expr('CAST(RAND() * 3650 AS INT)'))
    customer_df = customer_df.withColumn(f'{product}__time_since_last_close', expr('CAST(RAND() * 3650 AS INT)'))
    customer_df = customer_df.withColumn(f'{product}__time_since_first_new', expr('CAST(RAND() * 3650 AS INT)'))
    customer_df = customer_df.withColumn(f'{product}__age_of_oldest_active', expr('CAST(RAND() * 3650 AS INT)'))

# Generate Saving account features
customer_df = customer_df.withColumn('pit_overdraft_days', expr('CAST(RAND() * 30 AS INT)'))

# Generate Loans features
customer_df = customer_df.withColumn('static_amount_financed', expr('CAST(RAND() * 100000 AS INT)'))

# Generate Current account features
customer_df = customer_df.withColumn('tfeat_SOP_deposit_bal', expr('CAST(RAND() * 10000 AS INT)'))
customer_df = customer_df.withColumn('tfeat_SOP_deposit_bal_MA_1_3', expr('CAST(RAND() * 10000 AS INT)'))
customer_df = customer_df.withColumn('tfeat_SOP_deposit_bal_3MA_3M_growth', expr('RAND()'))
customer_df = customer_df.withColumn('tfeat_SOP_deposit_bal_3MA_YY_growth', expr('RAND()'))
customer_df = customer_df.withColumn('tfeat_SOP_deposit_bal_LAST_MO_VS_3MA', expr('RAND()'))

# Generate Deposits features
customer_df = customer_df.withColumn('tfeat_SOP_deposit_bal', expr('CAST(RAND() * 10000 AS INT)'))
customer_df = customer_df.withColumn('tfeat_SOP_deposit_bal_MA_1_3', expr('CAST(RAND() * 10000 AS INT)'))
customer_df = customer_df.withColumn('tfeat_SOP_deposit_bal_3MA_3M_growth', expr('RAND()'))
customer_df = customer_df.withColumn('tfeat_SOP_deposit_bal_3MA_YY_growth', expr('RAND()'))
customer_df = customer_df.withColumn('tfeat_SOP_deposit_bal_LAST_MO_VS_3MA', expr('RAND()'))

# Generate Bank online features
customer_df = customer_df.withColumn('tfeat_time_spent', expr('CAST(RAND() * 1440 AS INT)'))  # Minutes in a day
customer_df = customer_df.withColumn('tfeat_num_logins', expr('CAST(RAND() * 10 AS INT)'))
customer_df = customer_df.withColumn('tfeat_time_spent_lag1', expr('CAST(RAND() * 1440 AS INT)'))
customer_df = customer_df.withColumn('tfeat_num_logins_lag1', expr('CAST(RAND() * 10 AS INT)'))
customer_df = customer_df.withColumn('tfeat_cum_time_spent', expr('CAST(RAND() * 1440 * 365 AS INT)'))
customer_df = customer_df.withColumn('tfeat_cum_num_logins', expr('CAST(RAND() * 365 AS INT)'))

# Generate Loan history features
customer_df = customer_df.withColumn('LOAN_AMT', expr('CAST(RAND() * 100000 AS INT)'))
customer_df = customer_df.withColumn('Net_Prft', expr('CAST(RAND() * 1000 AS INT)'))
customer_df = customer_df.withColumn('Monthly_Income', expr('CAST(RAND() * 5000 AS INT)'))
customer_df = customer_df.withColumn('ReveNue', expr('CAST(RAND() * 5000 AS INT)'))

# Generate Credit cards features
customer_df = customer_df.withColumn('static_credit_limit', expr('CAST(RAND() * 10000 AS INT)'))

# Generate Insurance features
customer_df = customer_df.withColumn('gwp', expr('CAST(RAND() * 1000 AS INT)'))

# Generate Transaction features
transactions = ['_DR_total', '_DR_min', '_DR_max', '_DR_average', '_CR_total', '_CR_min', '_CR_max', '_CR_average', '_TR_count']

for transaction in transactions:
    customer_df = customer_df.withColumn(transaction, expr('CAST(RAND() * 10000 AS INT)'))

for transaction in transactions:
    customer_df = customer_df.withColumn(f'tfeat{transaction}_1_to_3m', expr('CAST(RAND() * 10000 AS INT)'))
    customer_df = customer_df.withColumn(f'tfeat{transaction}_4_to_6m', expr('CAST(RAND() * 10000 AS INT)'))
    customer_df = customer_df.withColumn(f'tfeat{transaction}_7_to_12m', expr('CAST(RAND() * 10000 AS INT)'))

# Save the DataFrame to a Parquet file
customer_df.write.mode("overwrite").parquet("data/customer360.parquet")


23/09/12 18:15:46 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 96.54% for 7 writers
23/09/12 18:15:46 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 84.47% for 8 writers
23/09/12 18:15:46 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 75.08% for 9 writers
23/09/12 18:15:46 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 67.58% for 10 writers
23/09/12 18:16:06 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 75.08% for 9 writers
23/09/12 18:16:06 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 84.47% for 8 writers
23/09/12 18:16:06 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,01

23/09/12 18:16:07 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 96.54% for 7 writers
23/09/12 18:16:07 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 84.47% for 8 writers
23/09/12 18:16:07 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 75.08% for 9 writers
23/09/12 18:16:07 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 67.58% for 10 writers
23/09/12 18:16:10 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 75.08% for 9 writers
23/09/12 18:16:10 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 84.47% for 8 writers
23/09/12 18:16:10 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,01

23/09/12 18:16:11 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 96.54% for 7 writers
23/09/12 18:16:11 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 84.47% for 8 writers
23/09/12 18:16:11 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 75.08% for 9 writers
23/09/12 18:16:11 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 67.58% for 10 writers
23/09/12 18:16:14 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 75.08% for 9 writers
23/09/12 18:16:14 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 84.47% for 8 writers
23/09/12 18:16:14 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,01

23/09/12 18:16:15 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 96.54% for 7 writers
23/09/12 18:16:15 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 84.47% for 8 writers
23/09/12 18:16:15 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 75.08% for 9 writers
23/09/12 18:16:15 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 67.58% for 10 writers
23/09/12 18:16:18 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 75.08% for 9 writers
23/09/12 18:16:18 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 84.47% for 8 writers
23/09/12 18:16:18 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,01

23/09/12 18:16:19 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 96.54% for 7 writers
23/09/12 18:16:19 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 84.47% for 8 writers
23/09/12 18:16:19 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 75.08% for 9 writers
23/09/12 18:16:19 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 67.58% for 10 writers
23/09/12 18:16:22 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 75.08% for 9 writers
23/09/12 18:16:22 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 84.47% for 8 writers
23/09/12 18:16:22 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,01

...

23/09/12 18:16:33 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 84.47% for 8 writers
23/09/12 18:16:33 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 96.54% for 7 writers


23/09/12 18:16:34 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 96.54% for 7 writers
23/09/12 18:16:34 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 84.47% for 8 writers
23/09/12 18:16:34 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 75.08% for 9 writers
23/09/12 18:16:34 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 67.58% for 10 writers
23/09/12 18:16:36 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 75.08% for 9 writers


23/09/12 18:16:37 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 84.47% for 8 writers


23/09/12 18:16:37 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 96.54% for 7 writers


23/09/12 18:16:38 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 96.54% for 7 writers
23/09/12 18:16:38 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 84.47% for 8 writers
23/09/12 18:16:38 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 75.08% for 9 writers
23/09/12 18:16:38 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 67.58% for 10 writers
23/09/12 18:16:41 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 75.08% for 9 writers
23/09/12 18:16:41 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 84.47% for 8 writers
23/09/12 18:16:41 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,01

In [13]:
df = spark.read.parquet('data/customer360.parquet')

In [14]:
df.groupBy(col('insurance__new_in_period')).agg(count(col('customer_no'))).show()

+------------------------+------------------+
|insurance__new_in_period|count(customer_no)|
+------------------------+------------------+
|                       1|             75119|
|                       0|           1424881|
+------------------------+------------------+



In [15]:
df.groupBy(col('savacc__new_in_period')).agg(count(col('customer_no'))).show()

+---------------------+------------------+
|savacc__new_in_period|count(customer_no)|
+---------------------+------------------+
|                    1|             75310|
|                    0|           1424690|
+---------------------+------------------+



In [16]:
df.groupBy(col('deposits__new_in_period')).agg(count(col('customer_no'))).show()

+-----------------------+------------------+
|deposits__new_in_period|count(customer_no)|
+-----------------------+------------------+
|                      1|             75095|
|                      0|           1424905|
+-----------------------+------------------+



In [17]:
df.groupBy(col('deposits__active_SOP')).agg(count(col('customer_no'))).show()

+--------------------+------------------+
|deposits__active_SOP|count(customer_no)|
+--------------------+------------------+
|                   1|           1357510|
|                   0|            142490|
+--------------------+------------------+



In [18]:
df.show(vertical=True)

-RECORD 0--------------------------------------------------------
 customer_no                              | 5488                 
 __PERIOD                                 | 2021-10-01           
 id_gender                                | 1                    
 customer_category_cleaned                | 1                    
 marital_status_cleaned                   | 0                    
 customer_governorate                     | 4                    
 age                                      | 43                   
 job_cleaned                              | 2                    
 local_branch                             | 96                   
 employed                                 | 0                    
 tenure                                   | 413                  
 customer_active_SOP                      | 1                    
 savacc__new_in_period                    | 0                    
 savacc__close_in_period                  | 0                    
 savacc__a